In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
rng = np.random.RandomState(50)
x = 10 * rng.rand(50)
y = 2 * x - 10 + rng.randn(50)
x
y
plt.scatter(x, y);

In [ ]:
plt.scatter(x,y);
plt.plot((0, 10), (-10, 10));
plt.plot((0, 10), (0 , 0));
plt.plot((0,10), (-10, 7));

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True) # Khởi tạo mô hình
# Training mô hình bằng cách gọi fit
# Fit nhận vào data và label để training, cú pháp x[:,np.newaxis] là để reshape lại x từ array sang ma trận
model.fit(x[:,np.newaxis], y)

xfit = np.array([0 , 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
yfit = model.predict(xfit[:, np.newaxis])
yfit
plt.scatter(x, y)
plt.plot(xfit,yfit);

In [ ]:
model.coef_

In [ ]:
model.intercept_

 # Cài đặt mô hình hồi quy tuyến tính
Về nguyên lý, hồi quy đa biến sẽ không khác mấy so với phần trình bày mô hình hồi quy đơn giản. Thế nên ở phần này, chúng ta sẽ thực hiện cài đặt chi tiết mô hình hồi quy tuyến tính để hiểu bản chất của nó, và đồng thời phần này cũng giới thiệu một thuật toán optimization (Tối ưu hàm mất mát - loss function) tên là Gradient Descent

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
rng = np.random.RandomState(1)
""" Tạo ra bộ dữ liệu 100 vector X, mỗi vector có 3 features
X1 = [ 1, 2, 3]
X2 = [ 3, 4, 5]
.....
Xn
X = [X1, X2, ..., Xn]
"""
X = 10* rng.rand(100, 3) # Tạo một ma trận 100 dòng, mỗi dòng 3 cột. Cú pháp rng.rand(100,3) nghĩa là: tạo 100 dòng, mỗi dòng 3 số lấy ngẫu nhiên(số thập phân) trong khoảng từ 0 -> 1. Nhân thêm 10 thì range là từ 0 -> 10,
Y = 0.5 + np.dot(X,[2, 3.5, 7]) # tạo 1 vector nhãn theo công thức: Y = 2*X1 + 3.5* X2 + 7*X3 + 0.5, bộ tham số là: W = [0.5, 2, 3.5, 7]

# Tạo 3D scatter plot - vẽ các điểm dữ liệu rời rạc
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# Scatter plot - Lấy X theo từng feature
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=Y, cmap='viridis')

ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('X3')
ax.set_title('Scatter Plot')
plt.show()

Bước tiếp theo, chúng ta sẽ sử dụng hàm loss MSE function để tính toán độ chênh lệch giữa giá trị dự đoán và giá trị thực tế. Sau đó tiến hành phương pháp gradient descent để cập nhật trọng số( weight) đến một ngưỡng nhất định( < epsilon)

In [ ]:
# kết hợp x, y thành một list tuples, xem như một dataset với data, label tương ứng
trainExample = list(zip(X,Y))
trainExample
# Khởi tạo hàm biến đổi điểm dữ liệu thành vector( X -> [X, 1])
# 1 ở đây là bias - intercept => [X1, X2, X3, 1]
def phi(x):
  return np.append(x,1)

In [ ]:
# Khởi tạo ma trận tham số - gọi chung là weight, w = [slope1, slope2, slope3, intercept]
def initialWeightVector():
  return np.zeros(4) # Trả về 1 vector gồm 4 số 0, [0, 0, 0, 0]

In [ ]:
# Khởi tạo hàm mất mát theo hàm MSE loss
def trainloss(w):
  total_loss = 0 # Khởi tạo 1 biến total_loss
  for x, y in trainExample:
    total_loss += (y - np.dot(phi(x), w))**2
  MSE = (1/len(trainExample)) * total_loss
  return MSE

In [ ]:
#Gradient of loss function
def gradientTrainloss(w):
  total_gradient = 0
  for x, y in trainExample:
    total_gradient += (y - np.dot(phi(x), w)) * (-phi(x))
  return (1/len(trainExample)) * total_gradient

In [ ]:
# Gradient descent algorithm
def gradientDescent(Loss, gradientTrainLoss, initialWeightVector):
  w = initialWeightVector() # Khởi tạo weight
  lr = 0.005
  for i in range(2500):
    value = Loss(w)
    gradient = gradientTrainLoss(w)
    w = w - lr * gradient
    print(f" epoch {i}: Weight Vector = {w}, Trainloss = {value}, Gradient of trainloss = {gradient}")
  return w

In [ ]:
# Goi hàm gradient để training
w = gradientDescent(trainloss, gradientTrainloss, initialWeightVector)

In [ ]:
yfit = np.dot(X, w[:-1]) + w[-1]

In [ ]:
y_predict = np.dot([2,3,4], w[:-1]) + w[-1]
y_predict

In [ ]:
# Tạo một meshgrid cho 3D plot
x1_range = np.linspace(0, 10, 30)
x2_range = np.linspace(0, 10, 30)
x1_mesh, x2_mesh = np.meshgrid(x1_range, x2_range)
x3_mesh = (0.5 - w[0] * x1_mesh - w[1] * x2_mesh - w[3]) / w[2]

# Hiển thị scatter các điểm dữ liệu
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=Y, cmap='viridis', label='Data Points')

# Plot, hiển thị mặt phẳng model đã tìm được

ax.plot_surface(x1_mesh, x2_mesh, x3_mesh, alpha=0.5, cmap='plasma', label='Model')

ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('X3')
ax.set_title('3D Scatter Plot with Fitted Model')

plt.show()